<a href="https://colab.research.google.com/github/SupunGurusinghe/sqlite-plus-colab/blob/main/sg_project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### The core ways to handle missing values should be familiar to all data scientists, a phrase which here means ‘if you aren’t familiar, you should memorise the following list’:


1. **Listwise deletion:** if a variable has so many missing cases that it appears useless, delete it.

2. **Casewise deletion:** if there are too many factors missing for a particular observation, delete it.

3. **Dummy Variable Adjustment:** if the variable is missing for a particular case, use an assumed value in its stead. Depending on the problem the median may appear the intuitive choice or a value that represents a ‘neutral’ setting.

4. **Imputation:** use an algorithm to fill in the value, from a simple random number at the most basic end of the spectrum, to a value imputed by its own model at the more complex end.



## Imports

In [32]:
import pandas as pd
from sqlite3 import connect

## DB Connection

In [33]:
conn = connect('test.db')

# **Basic NULL values handling**

## SQLite Creating a table

* employees : Table name
* id : employee id (primary key)
* f_name : first name of employee
* l_name : last name of employee
* title : title of employee
* NIC : national id number of employee (unique)

In [34]:
c = conn.cursor()

# dropping an existing table
c.execute("DROP TABLE IF EXISTS employees")

# create table
c.execute('''
  CREATE TABLE employees(
    id INT,
    f_name VARCHAR(50),
    l_name VARCHAR(50),
    title VARCHAR(10),
    age INT,
    wage INT,
    hire_date DATE,
    NIC VARCHAR(12)
  )
''')

employees = [(1, 'kavishka', 'tim', 'Mr', 22, 28, '2022-05-01', '200005303420'), 
             (1, 'Bill', 'Tibb', 'Mr', 61, 28, '2012-05-02', '900239889v'), 
             (3, 'Bill', 'Sadat', None, 18, 12, '2019-11-08', '640239889v'),
             (4, 'Christine', 'Riveles', None, 36, 20, '2018-03-30', '200014303420'),
             (5, 'David', 'Guerin', 'Honorable', 36, 20, '2018-03-30', '123456789v'),
             (6, 'David', 'Guerin', 'Honorable', 36, 20, '2018-03-30', '123456789v'),
             (None, 'David', 'Guerin', 'Honorable', 36, 20, '2018-03-30', '124097654988')]

c.executemany("INSERT INTO employees VALUES (?,?,?,?,?,?,?,?)", employees)

c.execute('SELECT * FROM employees')

results = c.fetchall()

for result in results:
  print(result)

c.close()

(1, 'kavishka', 'tim', 'Mr', 22, 28, '2022-05-01', '200005303420')
(1, 'Bill', 'Tibb', 'Mr', 61, 28, '2012-05-02', '900239889v')
(3, 'Bill', 'Sadat', None, 18, 12, '2019-11-08', '640239889v')
(4, 'Christine', 'Riveles', None, 36, 20, '2018-03-30', '200014303420')
(5, 'David', 'Guerin', 'Honorable', 36, 20, '2018-03-30', '123456789v')
(6, 'David', 'Guerin', 'Honorable', 36, 20, '2018-03-30', '123456789v')
(None, 'David', 'Guerin', 'Honorable', 36, 20, '2018-03-30', '124097654988')




---



## Identify `NULL` values

Add 1 for the column if null value appears.

In [102]:
c = conn.cursor()

c.execute('''
  SELECT 
    SUM(CASE WHEN id IS NULL THEN 1 ELSE 0 END) AS id ,
    SUM(CASE WHEN f_name IS NULL THEN 1 ELSE 0 END) AS f_name,
    SUM(CASE WHEN l_name IS NULL THEN 1 ELSE 0 END) AS l_name,
    SUM(CASE WHEN title IS NULL THEN 1 ELSE 0 END) AS title,
    SUM(CASE WHEN age IS NULL THEN 1 ELSE 0 END) AS age,
    SUM(CASE WHEN wage IS NULL THEN 1 ELSE 0 END) AS wage,
    SUM(CASE WHEN hire_date IS NULL THEN 1 ELSE 0 END) AS hire_date
  FROM employees
''')

results = c.fetchall()

for result in results:
  print(result)

c.close()

(1, 0, 0, 2, 0, 0, 0)


## Selecting rows having `NULL` value for a particular column

In [103]:
c = conn.cursor()

c.execute('''
  SELECT *
  FROM employees
  WHERE title IS NULL
''')

results = c.fetchall()

for result in results:
  print(result)

c.close()

(3, 'Bill', 'Sadat', None, 18, 12, '2019-11-08')
(4, 'Christine', 'Riveles', None, 36, 20, '2018-03-30')


## Selecting rows having `NULL` value for any of the columns

In [104]:
c = conn.cursor()

c.execute('''
  SELECT *
  FROM employees
  WHERE (id || f_name || l_name || title || age || wage || hire_date) IS NULL
''')

results = c.fetchall()

for result in results:
  print(result)

c.close()

(3, 'Bill', 'Sadat', None, 18, 12, '2019-11-08')
(4, 'Christine', 'Riveles', None, 36, 20, '2018-03-30')
(None, 'David', 'Guerin', 'Honorable', 36, 20, '2018-03-30')




---

1. There are many missing values (NULLs) of a column, but the columns 
itself is not of interest from analysis point of view

2. There may be no missing value of a column, but it is excluded from the analysis we are preparing the Dataset for

3. There may be very few missing values of column, but dropping the field (column) is better than replacing those values



## Deleting rows where column value is `NULL`

In [35]:
c = conn.cursor()

c.execute('''
  DELETE FROM employees WHERE id IS NULL
''')

c.close()

## Dropping a column when all rows have `NULL` for that column

In [5]:
# If all null
c = conn.cursor()

c.execute('''SELECT id FROM employees GROUP BY id''')

results = c.fetchall()

for result in results:
  print(result)

c.close()

(None,)
(1,)
(3,)
(4,)
(5,)


In [107]:
# If all null
c = conn.cursor()

c.execute('''SELECT DISTINCT id FROM employees''')

results = c.fetchall()

for result in results:
  print(result)

c.close()

(1,)
(2,)
(3,)
(4,)
(5,)


In [108]:
# If zero (0)
c = conn.cursor()

c.execute('''SELECT count(id) FROM employees WHERE id IS NOT NULL''')

results = c.fetchall()

for result in results:
  print(result)

c.close()

(5,)


In [121]:
#  if you only get back just NULL (or 0 for that last one)
c = conn.cursor()

c.execute('''CREATE TEMPORARY TABLE t1_backup(f_name, l_name, title, age, wage, hire_date)''')
c.execute('''INSERT INTO t1_backup SELECT f_name, l_name, title, age, wage, hire_date FROM employees''')
c.execute('''DROP TABLE employees''')
c.execute('''CREATE TABLE employees(f_name, l_name, title, age, wage, hire_date)''')
c.execute('''INSERT INTO employees SELECT f_name, l_name, title, age, wage, hire_date FROM t1_backup''')
c.execute('''DROP TABLE t1_backup''')

c.execute('''
  SELECT *
  FROM employees
''')
results = c.fetchall()

for result in results:
  print(result)

c.close()

('kavishka', 'tim', 'Mr', 22, 28, '2022-05-01')
('Bill', 'Tibb', 'Mr', 61, 28, '2012-05-02')
('Bill', 'Sadat', None, 18, 12, '2019-11-08')
('Christine', 'Riveles', None, 36, 20, '2018-03-30')
('David', 'Guerin', 'Honorable', 36, 20, '2018-03-30')
('David', 'Guerin', 'Honorable', 36, 20, '2018-03-30')


## Replace `NULL` values with a sentinel (standard value)

In [122]:
c = conn.cursor()

c.execute('''
  SELECT 
    f_name,
    l_name,
    CASE WHEN title IS NULL THEN 'Honorable' ELSE title END AS NewTitle
  FROM employees
''')

results = c.fetchall()

for result in results:
  print(result)

c.close()

('kavishka', 'tim', 'Mr')
('Bill', 'Tibb', 'Mr')
('Bill', 'Sadat', 'Honorable')
('Christine', 'Riveles', 'Honorable')
('David', 'Guerin', 'Honorable')
('David', 'Guerin', 'Honorable')


# **Advance `NULL` values handling**

## Replace by an statistical technique such as mean

Let us now discuss another form of data wrangling by imputing (replacing) missing values with the help of Mean Method to improve the data quality.  This method requires us to calculate statistical mean value of the series of the dataset to impute (replace) missing values.

In [123]:
c = conn.cursor()

# dropping an existing table
c.execute("DROP TABLE IF EXISTS house_price")

# create table
c.execute('''
  CREATE TABLE house_price(
    id INT,
    country VARCHAR(50),
    city VARCHAR(50),
    price DOUBLE,
    a DOUBLE,
    b DOUBLE,
    c DOUBLE
  )
''')

house_price = [(1, 'USA', 'LA', 1000000.00, 1, 3, 5), 
             (2, 'UK', 'London', 400000.00, None, 5, 7), 
             (3, 'USA', 'LA', 850000.00, 9, None, None),
             (4, 'USA', 'LA', None, 12, 4, 9),
             (5, 'USA', 'LA', 900000.00, 2, 6, 1),
             (6, 'UK', 'London', 550000.00, None, 4, 8),
             (7, 'USA', 'LA', 1000000.00, 8, 8, 8), 
             (8, 'UK', 'London', 400000.00, 1, 4, 9), 
             (9, 'USA', 'LA', 850000.00, 4, 4, 5),
             (10, 'USA', 'LA', 1050000.00, None, None, None),
             (11, 'USA', 'LA', 900000.00, 3, 8.5, 9),
             (12, 'UK', 'London', None, 10, 7, None)]

c.executemany("INSERT INTO house_price VALUES (?,?,?,?,?,?,?)", house_price)

c.execute('SELECT * FROM house_price')

results = c.fetchall()

for result in results:
  print(result)

c.close()

(1, 'USA', 'LA', 1000000.0, 1.0, 3.0, 5.0)
(2, 'UK', 'London', 400000.0, None, 5.0, 7.0)
(3, 'USA', 'LA', 850000.0, 9.0, None, None)
(4, 'USA', 'LA', None, 12.0, 4.0, 9.0)
(5, 'USA', 'LA', 900000.0, 2.0, 6.0, 1.0)
(6, 'UK', 'London', 550000.0, None, 4.0, 8.0)
(7, 'USA', 'LA', 1000000.0, 8.0, 8.0, 8.0)
(8, 'UK', 'London', 400000.0, 1.0, 4.0, 9.0)
(9, 'USA', 'LA', 850000.0, 4.0, 4.0, 5.0)
(10, 'USA', 'LA', 1050000.0, None, None, None)
(11, 'USA', 'LA', 900000.0, 3.0, 8.5, 9.0)
(12, 'UK', 'London', None, 10.0, 7.0, None)


In [124]:
c = conn.cursor()

c.execute('''
  SELECT 
    h.country, 
    h.city,
    COALESCE(h.price, n.newprice) AS price_new
  FROM house_price h, (SELECT s.city, AVG(s.price) AS newprice
        FROM house_price s
        GROUP BY s.city) n
  WHERE h.city = n.city
''')

results = c.fetchall()

for result in results:
  print(result)

c.close()

('USA', 'LA', 1000000.0)
('UK', 'London', 400000.0)
('USA', 'LA', 850000.0)
('USA', 'LA', 935714.2857142857)
('USA', 'LA', 900000.0)
('UK', 'London', 550000.0)
('USA', 'LA', 1000000.0)
('UK', 'London', 400000.0)
('USA', 'LA', 850000.0)
('USA', 'LA', 1050000.0)
('USA', 'LA', 900000.0)
('UK', 'London', 450000.0)


`Note: Instead of AVG(), use functions such as MAX(), MIN() as necessary`

## Handling Missing Values in Time-series with SQL

In [6]:
c = conn.cursor()

# dropping an existing table
c.execute("DROP TABLE IF EXISTS hourly_machine_data")

# create table
c.execute('''
  CREATE TABLE hourly_machine_data(
    Observation_datetime DATETIME,
    Machine_ID INT,
    Casing_Temperature_F INT,
    Bearing_Temperature_F INT,
    Flywheel_rpm INT,
    alarm_status VARCHAR(15),
    Flywheel_rpm_2 INT,
    Filter_airflow DOUBLE,
    TARGET_failure_in_next_90 INT
  )
''')

hourly_machine_data = [('2022-01-01 1:00 AM',1,84,131,2374,'normal',2171,1.1,0),
                ('2022-01-01 2:00 AM',1,None,132,1587,'normal',1877,1.5,0),
                ('2022-01-01 3:00 AM',1,85,133,1206,'normal',1296,1.8,0),
                ('2022-01-01 4:00 AM',1,None,None,2181,'normal',1879,2,0),
                ('2022-01-01 5:00 AM',1,None,134,1271,'normal',2170,1.7,0),
                ('2022-01-01 6:00 AM',1,None,None,1508,'normal',1556,1.3,0),
                ('2022-01-01 7:00 AM',1,86,135,1298,'normal',1749,1.9,0),
                ('2022-01-01 8:00 AM',1,87,None,1327,'normal',2058,None,0),
                ('2022-01-01 9:00 AM',1,88,136,1978,'normal',1501,1.3,0),
                ('2022-01-01 10:00 AM',1,None,None,2131,'normal',1952,1.4,0),
                ('2022-01-01 11:00 AM',1,None,137,1611,'normal',2049,1,0),
                ('2022-01-01 12:00 PM',1,89,None,1388,'normal',2400,2,0),
                ('2022-01-01 1:00 PM',1,90,138,1596,'normal',1453,1.2,0),
                ('2022-01-01 2:00 PM',1,91,None,1911,'warning',1680,None,0),
                ('2022-01-01 3:00 PM',1,None,139,2368,'warning',1496,1,0),
                ('2022-01-01 4:00 PM',1,None,None,2055,'warning',1574,1.2,0),
                ('2022-01-01 5:00 PM',1,92,140,1961,'warning',2252,1.9,0),
                ('2022-01-01 6:00 PM',1,93,None,2314,'warning',1860,2,0),
                ('2022-01-01 7:00 PM',1,None,141,2046,'warning',2378,1.5,0),
                ('2022-01-01 8:00 PM',1,92,None,1880,'warning',1364,1.2,0),
                ('2022-01-01 9:00 PM',1,91,142,1289,'warning',2174,1.5,0),
                ('2022-01-01 10:00 PM',1,90,141,1648,'normal',1928,1.9,0),
                ('2022-01-01 11:00 PM',1,89,140,1225,'normal',2035,1.8,0),
                ('2022-01-02 12:00 AM',1,88,139,1403,'normal',2139,1.1,0),
                ('2022-01-02 1:00 AM',1,87,138,1381,'normal',1230,None,0),
                ('2022-01-02 2:00 AM',1,86,137,1720,'normal',1203,1.6,0),
                ('2022-01-02 3:00 AM',1,85,136,2392,'normal',2148,1.3,0),
                ('2022-01-02 4:00 AM',1,84,135,1956,'normal',2073,1.2,0)]

c.executemany("INSERT INTO hourly_machine_data VALUES (?,?,?,?,?,?,?,?,?)", hourly_machine_data)

c.execute('SELECT * FROM hourly_machine_data')

results = c.fetchall()

for result in results:
  print(result)

c.close()

('2022-01-01 1:00 AM', 1, 84, 131, 2374, 'normal', 2171, 1.1, 0)
('2022-01-01 2:00 AM', 1, None, 132, 1587, 'normal', 1877, 1.5, 0)
('2022-01-01 3:00 AM', 1, 85, 133, 1206, 'normal', 1296, 1.8, 0)
('2022-01-01 4:00 AM', 1, None, None, 2181, 'normal', 1879, 2.0, 0)
('2022-01-01 5:00 AM', 1, None, 134, 1271, 'normal', 2170, 1.7, 0)
('2022-01-01 6:00 AM', 1, None, None, 1508, 'normal', 1556, 1.3, 0)
('2022-01-01 7:00 AM', 1, 86, 135, 1298, 'normal', 1749, 1.9, 0)
('2022-01-01 8:00 AM', 1, 87, None, 1327, 'normal', 2058, None, 0)
('2022-01-01 9:00 AM', 1, 88, 136, 1978, 'normal', 1501, 1.3, 0)
('2022-01-01 10:00 AM', 1, None, None, 2131, 'normal', 1952, 1.4, 0)
('2022-01-01 11:00 AM', 1, None, 137, 1611, 'normal', 2049, 1.0, 0)
('2022-01-01 12:00 PM', 1, 89, None, 1388, 'normal', 2400, 2.0, 0)
('2022-01-01 1:00 PM', 1, 90, 138, 1596, 'normal', 1453, 1.2, 0)
('2022-01-01 2:00 PM', 1, 91, None, 1911, 'warning', 1680, None, 0)
('2022-01-01 3:00 PM', 1, None, 139, 2368, 'warning', 1496, 1.0, 0

In [8]:
c = conn.cursor()

c.execute('''
SELECT 
  Machine_ID, 
  Observation_datetime, 
  LAST_VALUE(
    CASING_TEMPERATURE_F ignore NULLS
  ) OVER (
    PARTITION BY MACHINE_ID 
    ORDER BY 
      OBSERVATION_DATETIME ROWS BETWEEN UNBOUNDED PRECEDING 
      AND CURRENT ROW
  ) AS LATEST_CASING_TEMPERATURE_F, 
  LAST_VALUE(
    BEARING_TEMPERATURE_F ignore NULLS
  ) OVER (
    PARTITION BY MACHINE_ID 
    ORDER BY 
      OBSERVATION_DATETIME ROWS BETWEEN UNBOUNDED PRECEDING 
      AND CURRENT ROW
  ) AS LATEST_BEARING_TEMPERATURE_F, 
  LAST_VALUE(FLYWHEEL_RPM ignore NULLS) OVER (
    PARTITION BY MACHINE_ID 
    ORDER BY 
      OBSERVATION_DATETIME ROWS BETWEEN UNBOUNDED PRECEDING 
      AND CURRENT ROW
  ) AS LATEST_FLYWHEEL_RPM
FROM hourly_machine_data
''')

results = c.fetchall()

for result in results:
  print(result)

c.close()

OperationalError: ignored

In [ ]:
c = conn.cursor()

c.execute('''
SELECT 
  Machine_ID, 
  Observation_datetime, 
  LAST_VALUE(
    COALESCE(Casing_Temperature_F)
  ) OVER (
    PARTITION BY Machine_ID 
    ORDER BY 
      Observation_datetime ROWS BETWEEN UNBOUNDED PRECEDING 
      AND CURRENT ROW
  ) AS LATEST_CASING_TEMPERATURE_F, 
  LAST_VALUE(
    COALESCE(Bearing_Temperature_F)
  ) OVER (
    PARTITION BY Machine_ID 
    ORDER BY 
      Observation_datetime ROWS BETWEEN UNBOUNDED PRECEDING 
      AND CURRENT ROW
  ) AS LATEST_BEARING_TEMPERATURE_F, 
  LAST_VALUE(
    COALESCE(Flywheel_rpm)
    ) OVER (
    PARTITION BY Machine_ID 
    ORDER BY 
      Observation_datetime ROWS BETWEEN UNBOUNDED PRECEDING 
      AND CURRENT ROW
  ) AS LATEST_FLYWHEEL_RPM
FROM hourly_machine_data
''')

results = c.fetchall()

for result in results:
  print(result)

c.close()

## **Uniquness violations in record level**


---



## Identify uniqueness violations for a key

In [43]:
c = conn.cursor()

c.execute('''
  SELECT *, COUNT(*) AS duplicates
  FROM employees
  GROUP BY id
  HAVING COUNT(*) > 1
''')

results = c.fetchall()

for result in results:
  print(result)

c.close()

(1, 'Bill', 'Tibb', 'Mr', 61, 28, '2012-05-02', '900239889v', 2)


## Identify uniqueness violations (Duplicate all columns except primary key)

In [36]:
c = conn.cursor()

c.execute('''
  SELECT *, COUNT(*) AS duplicates
  FROM employees
  GROUP BY f_name, l_name, title, age, wage, hire_date, NIC
  HAVING COUNT(*) > 1
''')

results = c.fetchall()

for result in results:
  print(result)

c.close()

(6, 'David', 'Guerin', 'Honorable', 36, 20, '2018-03-30', '123456789v', 2)


## Identify uniqueness violations for a key - Display all

In [41]:
c = conn.cursor()

c.execute('''
  SELECT a.*
  FROM employees a, (SELECT *, COUNT(*)
        FROM employees
        GROUP BY id
        HAVING COUNT(*) > 1) b
  WHERE a.id = b.id
''')

results = c.fetchall()

for result in results:
  print(result)

c.close()

(1, 'kavishka', 'tim', 'Mr', 22, 28, '2022-05-01', '200005303420')
(1, 'Bill', 'Tibb', 'Mr', 61, 28, '2012-05-02', '900239889v')


## Identify uniqueness violations (Duplicate all columns except primary key) - Display all

In [44]:
c = conn.cursor()

c.execute('''
  SELECT a.*
  FROM employees a, (SELECT *, COUNT(*) AS duplicates
        FROM employees
        GROUP BY f_name, l_name, title, age, wage, hire_date, NIC
        HAVING COUNT(*) > 1) b
  WHERE a.f_name = b.f_name and 
        a.l_name = b.l_name and 
        a.title = b.title and 
        a.age = b.age and 
        a.wage = b.wage and 
        a.hire_date = b.hire_date and 
        a.NIC = b.NIC
''')

results = c.fetchall()

for result in results:
  print(result)

c.close()

(5, 'David', 'Guerin', 'Honorable', 36, 20, '2018-03-30', '123456789v')
(6, 'David', 'Guerin', 'Honorable', 36, 20, '2018-03-30', '123456789v')


## Removing duplicates

In [45]:
c = conn.cursor()

c.execute('''
  SELECT *, COUNT(id) AS duplicates
  FROM employees
  GROUP BY f_name, l_name, title, age, wage, hire_date, NIC
  HAVING COUNT(*) > 1
''')

results = c.fetchall()

for result in results:
  c.execute("DELETE FROM employees WHERE id = ?", (result[0],))
  
c.execute('''
  SELECT *
  FROM employees
''')

results = c.fetchall()

for result in results:
  print(result)

c.close()

(1, 'kavishka', 'tim', 'Mr', 22, 28, '2022-05-01', '200005303420')
(1, 'Bill', 'Tibb', 'Mr', 61, 28, '2012-05-02', '900239889v')
(3, 'Bill', 'Sadat', None, 18, 12, '2019-11-08', '640239889v')
(4, 'Christine', 'Riveles', None, 36, 20, '2018-03-30', '200014303420')
(5, 'David', 'Guerin', 'Honorable', 36, 20, '2018-03-30', '123456789v')
